# Chapter 5: Formatting Output and Speaking for Claude

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# !pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic
import os

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
# doesn't work
# %store -r API_KEY
# %store -r MODEL_NAME

API_KEY = os.environ.get("ANTHROPIC_API_KEY")
MODEL_NAME = "claude-3-haiku-20240307"

client = anthropic.Anthropic(api_key=API_KEY)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

**Claude can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also ask Claude to **use XML tags to make its output clearer and more easily understandable** to humans.

### Examples

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling Claude to put the poem in XML tags**.

In [2]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Fluffy, twitching ears,
Hopping through the verdant field,
Rabbit's gentle grace.
</haiku>


Why is this something we'd want to do? Well, having the output in **XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags**.

An extension of this technique is to **put the first XML tag in the `assistant` turn. When you put text in the `assistant` turn, you're basically telling Claude that Claude has already said something, and that it should continue from that point onward. This technique is called "speaking for Claude" or "prefilling Claude's response."

Below, we've done this with the first `<haiku>` XML tag. Notice how Claude continues directly from where we left off.

In [3]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly by my side,
Cat, my loyal friend.
</haiku>


Claude also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill Claude's response with the opening bracket, `{`}.

In [4]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly by my side,",
  "third_line": "Captivating cat."
}


Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [7]:
# New system prompt
SYSTEM = "You are a songwriter and pop music critique who knows popular artists and how to write good lyrics."

# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "Bruno Mars song"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more like a {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, system_prompt=SYSTEM, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more like a Bruno Mars song. Write the new version in <Bruno Mars song_email> XML tags.

ASSISTANT TURN
<Bruno Mars song_email>

------------------------------------- Claude's response -------------------------------------

Hey Zack, I'm just checkin' in, 
Tryna get the latest spin,
On that prompt you were supposed to write.
I know you've been workin' hard,
Puttin' in that overtime card,
But I'm waitin' to see the fruits of your might.

So let me know what's goin' on,
Is the work almost done and gone?
I'm eager to see what you've come up with, my friend.
Just shoot me a quick update,
I promise I won't hesitate,
To give you a pat on the back in the end.

Alright Zack, I'll be waitin',
For that message you'll be sendin',
Hopin' it's good 

#### Bonus lesson

If you are calling Claude through the API, you can pass the closing XML tag to the `stop_sequences` parameter to get Claude to stop sampling once it emits your desired tag. This can save money and time-to-last-token by eliminating Claude's concluding remarks after it's already given you the answer you care about.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Change the `PREFILL` variable to **compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [13]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = "Steph Curry, who plays for"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN
Steph Curry, who plays for

------------------------------------- Claude's response -------------------------------------
 the Golden State Warriors, is often considered one of the best basketball players of all time. Some key reasons why Curry is viewed as one of the greatest:

- Shooting Ability: Curry is widely regarded as the greatest shooter in NBA history. He has revolutionized the game with his incredible range and accuracy from the 3-point line.

- Impact on the Game: Curry's shooting ability has transformed the way the game is played, with teams now emphasizing 3-point shooting more than ever before. He has raised the bar for what is possible in the sport.

- Individual Accolades: Curry is a 4-time NBA champion, 2-time NBA MVP, 8-time NBA All-Star, and has led the league in 3-po

❓ If you want a hint, run the cell below!

In [14]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

The grading function for this exercise is looking for a response that includes the word "Warrior".
Write more words in Claude's voice to steer Claude to act the way you want it to. For instance, instead of "Stephen Curry is the best because," you could write "Stephen Curry is the best and here are three reasons why. 1:


### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [15]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write two haikus about {ANIMAL}. Put each one in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write two haikus about cats. Put each one in <haiku> tags.

ASSISTANT TURN
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly in the sun,
Cats rule the domain.
</haiku>

<haiku>
Whiskers twitching, eyes
Watchful, stalking unseen prey,
Cats, masters of stealth.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [16]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

The grading function looks for a response of over 5 lines in length that includes the words "cat" and "<haiku>".
Start simple. Currently, the prompt asks Claude for one haiku. You can change that and ask for two (or even more). Then if you run into formatting issues, change your prompt to fix that after you've already gotten Claude to write more than one haiku.


### Exercise 5.3 - Two Haikus, Two Animals
Modify the `PROMPT` below so that **Claude produces two haikus about two different animals**. Use `{ANIMAL1}` as a stand-in for the first substitution, and `{ANIMAL2}` as a stand-in for the second substitution.

In [19]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

ANIMALS = {"Cat", "Dog", "Ferret"}

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about each animal in {ANIMALS}. Put each one in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about each animal in {'Cat', 'Dog', 'Ferret'}. Put each one in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
Here are haikus about each animal in the set {'Cat', 'Dog', 'Ferret'}:

<haiku>
Feline grace and poise,
Purring softly by the fire,
Mysterious cat.
</haiku>

<haiku>
Loyal companion,
Wagging tail, playful spirit,
Faithful, loving dog.
</haiku>

<haiku>
Curious explorer,
Nimble, agile, and quick-witted,
Mischievous ferret.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [18]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

The grading function in this exercise is looking for a response that contains the words "tail", "cat", and "<haiku>".
It's helpful to break this exercise down to several steps.								
1.	Modify the initial prompt template so that Claude writes two poems.							
2.	Give Claude indicators as to what the poems will be about, but instead of writing in the subjects directly (e.g., dog, cat, etc.), replace those subjects with the keywords "{ANIMAL1}" and "{ANIMAL2}".							
3.	Run the prompt and make sure that the full prompt with variable substitutions has all the words correctly substituted. If not, check to make sure your {bracket} tags are spelled correctly and formatted correctly with single moustache brackets.


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [5]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "indonesian"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
# PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
# print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more indonesian. Write the new version in <indonesian_email> XML tags.

ASSISTANT TURN

------------------------------------- Claude's response -------------------------------------
Here is the email in Indonesian:

<indonesian_email>
Hai Zack,
Saya hanya mengingatkan Anda untuk memberikan update singkat tentang prompt yang seharusnya Anda tulis.
Terima kasih,
</indonesian_email>


In [8]:
# SYSTEM = "You are an early 17th century English poet from Bristol. You like writing poems about nature, animals, and the wonders of the natural world."

# First input variable
IDEA = "A love affair between two opposites"

# Second input variable
ADJECTIVE = "Shakespeare"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Write a poem not more than 10 lines about {IDEA} in the style of {ADJECTIVE}. Put the poem in <poem> xml tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"Remember, you are a poet. <poem>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, system_prompt=SYSTEM, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Write a poem not more than 10 lines about A love affair between two opposites in the style of Shakespeare. Put the poem in <poem> xml tags.

ASSISTANT TURN
Remember, you are a poet. <poem>

------------------------------------- Claude's response -------------------------------------


Two souls, so different, drawn together,
A love affair, a tale to remember.
The hawk and the dove, the fire and the ice,
Opposites attract, a bond so precise.

Soaring high, the hawk's fierce gaze,
Gentle dove, her soft wings ablaze.
Passion ignites, a tempestuous dance,
Flames of desire, a captivating trance.

Though worlds apart, their hearts entwine,
A love so pure, a bond divine.
</poem>


In [9]:
# SYSTEM = "You are an early 20th century American poet from Massachusetts. You like writing poems about nature, animals, and the wonders of the natural world."

# First input variable
IDEA = "Woods during the fall season"

# Second input variable
ADJECTIVE = "Robert Frost"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Write a poem about woods during the fall season"

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"Remember, you are a poet. <poem>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Write a poem about woods during the fall season

ASSISTANT TURN
Remember, you are a poet. <poem>

------------------------------------- Claude's response -------------------------------------
Here is a poem about woods during the fall season:

Autumn's Embrace

The woods come alive with vibrant hues,
Painting the landscape in nature's muse.
Leaves of gold, crimson, and amber glow,
As the gentle breeze makes them softly flow.

The air grows crisp, a chill in the air,
Inviting us to bundle up and linger there.
Beneath our feet, a carpet of fallen leaves,
Crunching with each step, a symphony it weaves.

The trees stand tall, their branches bare,
Reaching up to the sky, a sight so fair.
Squirrels scurry 'round, gathering their stash,
Preparing for winter's icy lash.

In the woods, a tranquil, serene retreat,
Where autumn's embrace makes our hearts beat.
A time of transition, 